In [1]:
from cartiflette.utils import import_yaml_config
from cartiflette.config import FS
from cartiflette.s3 import upload_s3_raw
from cartiflette.download.download import _download_sources
from cartiflette.utils import create_path_bucket, official_epsg_codes
from cartiflette.config import PATH_WITHIN_BUCKET
from cartiflette.utils import hash_file

intermediate_dir="temp"
path_within_bucket="test-clean"
fs=FS
provider="IGN"
source="EXPRESS-COG-CARTO-TERRITOIRE"
year=2022
dataset_family="ADMINEXPRESS"
territory="metropole"
borders="COMMUNE"
path_within_bucket=PATH_WITHIN_BUCKET
vectorfile_format="shp"
bucket="projet-cartiflette"

In [2]:
local_dir = intermediate_dir
format_intermediate = "geojson"

yaml = import_yaml_config()

list_territories = yaml["IGN"]["ADMINEXPRESS"]["EXPRESS-COG-TERRITOIRE"][
        "territory"
].keys()

In [3]:
    from collections import OrderedDict 
    kwargs = OrderedDict()
    items = [
        ("sources", source),
        ("territories", territory),
        ("years", year),
        ("providers", provider),
        ("dataset_families", dataset_family),
    ]
    for key, val in items:
        if isinstance(val, str) or isinstance(val, int):
            kwargs[key] = [val]
        elif not val:
            kwargs[key] = [None]
        elif isinstance(val, list) or isinstance(val, tuple) or isinstance(val, set):
            kwargs[key] = list(val)
    kwargs

OrderedDict([('sources', ['EXPRESS-COG-CARTO-TERRITOIRE']),
             ('territories', ['metropole']),
             ('years', [2022]),
             ('providers', ['IGN']),
             ('dataset_families', ['ADMINEXPRESS'])])

In [4]:
from cartiflette.download.dataset import Dataset

datafile = Dataset(
                dataset_family,
                source,
                year,
                provider,
                territory,
                bucket,
                path_within_bucket,
            )
datafile

<Dataset IGN ADMINEXPRESS EXPRESS-COG-CARTO-TERRITOIRE metropole 2022>

In [5]:
from cartiflette.download.scraper import MasterScraper

hash = datafile.md5
url = datafile.get_path_from_provider()
with MasterScraper() as s:
    result = s.download_unpack(datafile)

In [10]:
x = _download_sources(
    upload=True,
    providers=provider,
    dataset_families=dataset_family,
    sources=source,
    territories=territory,
    years=year,
    path_within_bucket=path_within_bucket
)

In [9]:
x

{'IGN': {'ADMINEXPRESS': {'EXPRESS-COG-CARTO-TERRITOIRE': {'metropole': {2022: {'downloaded': True,
      'paths': {'CHFLIEU_COMMUNE': ['projet-cartiflette/production/provider=IGN/dataset_family=ADMINEXPRESS/source=EXPRESS-COG-CARTO-TERRITOIRE/year=2022/administrative_level=None/crs=2154/origin=raw/vectorfile_format=shp/territory=metropole/simplification=0/CHFLIEU_COMMUNE.shp'],
       'COMMUNE': ['projet-cartiflette/production/provider=IGN/dataset_family=ADMINEXPRESS/source=EXPRESS-COG-CARTO-TERRITOIRE/year=2022/administrative_level=None/crs=2154/origin=raw/vectorfile_format=shp/territory=metropole/simplification=0/COMMUNE.shp'],
       'COLLECTIVITE_TERRITORIALE': ['projet-cartiflette/production/provider=IGN/dataset_family=ADMINEXPRESS/source=EXPRESS-COG-CARTO-TERRITOIRE/year=2022/administrative_level=None/crs=2154/origin=raw/vectorfile_format=shp/territory=metropole/simplification=0/COLLECTIVITE_TERRITORIALE.shp'],
       'COMMUNE_ASSOCIEE_OU_DELEGUEE': ['projet-cartiflette/producti